# Object Detection using TensorFlow - Webcam

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from utils import label_map_util
from utils import visualization_utils as vis_util
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

In [3]:
# path to the download url 
download_url = 'http://download.tensorflow.org/models/object_detection/'
# choosing a model to downlaod
model = 'ssd_mobilenet_v1_coco_11_06_2017'
model_tar = model + '.tar.gz'
# path to the pb file
path = model + '/frozen_inference_graph.pb'
# path to the labels of the pretrained dataset
label_path = os.path.join('data', 'mscoco_label_map.pbtxt')
# 90 classes
classes_num = 90

In [4]:
# opens the tar file and downloads the model to our system
opener = urllib.request.URLopener()
opener.retrieve(download_url + model_tar, model_tar)
file = tarfile.open(model_tar)
print(file)
for each_file in file.getmembers():
    each_file_name = os.path.basename(each_file.name)
    if 'frozen_inference_graph.pb' in each_file_name:
        file.extract(each_file, os.getcwd())

In [5]:
# loading a frozen tensorflow model into memory
graph_detection = tf.Graph()
with graph_detection.as_default():
    graph_def = tf.GraphDef()
    with tf.gfile.GFile(path, 'rb') as fid:
        graph_serialized = fid.read()
        graph_def.ParseFromString(graph_serialized)
        tf.import_graph_def(graph_def, name='')

In [6]:
# loading labels and their mappings
label_map = label_map_util.load_labelmap(label_path)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=classes_num, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [7]:
import cv2
cap=cv2.VideoCapture(0)
ret = True
with graph_detection.as_default():
    with tf.Session(graph=graph_detection) as sess:
   
     while(ret):
        ret,image_np=cap.read()
        image_np_expanded = np.expand_dims(image_np, axis=0)
        image_tensor = graph_detection.get_tensor_by_name('image_tensor:0')
        
      # Each box represents a part of the image where a particular object was detected.
        boxes = graph_detection.get_tensor_by_name('detection_boxes:0')
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
        scores = graph_detection.get_tensor_by_name('detection_scores:0')
        classes = graph_detection.get_tensor_by_name('detection_classes:0')
        num_detections = graph_detection.get_tensor_by_name('num_detections:0')
      # Actual detection.
        (boxes, scores, classes, num_detections) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8)
        cv2.imshow('image',cv2.resize(image_np,(1280,960)))
        if cv2.waitKey(25) & 0xFF==ord('q'):
            break
            cv2.destroyAllWindows()
            cap.release()

KeyboardInterrupt: 